#### 클러스터링

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import umap
import hdbscan
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from tqdm import tqdm
import os

# 경로 설정
INPUT_CSV_PATH = "../data/processed/news_predict_result.csv"
OUTPUT_DIR = "../data/processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

FINAL_OUTPUT_CSV = os.path.join(OUTPUT_DIR, "news_cluster.csv")
EMBEDDING_FILE_PATH = os.path.join(OUTPUT_DIR, "full_article_embeddings.npz")

# 하이퍼파라미터
MERGE_THRESHOLD = 0.80
REASSIGN_THRESHOLD = 0.70 

def get_representative_doc(vectors, texts_in_cluster):
    """대표 문장으로 content를 반환"""
    if not texts_in_cluster or len(vectors) == 0: return None
    centroid = np.mean(vectors, axis=0)
    distances = cosine_distances([centroid], vectors)[0]
    return texts_in_cluster[np.argmin(distances)]

def merge_clusters(df_group, group_embeddings, merge_threshold, cluster_col='cluster_id'):
    valid_clusters = df_group[df_group[cluster_col] != -1]
    if valid_clusters.empty or valid_clusters[cluster_col].nunique() < 2:
        return df_group
    
    centroids = {
        cid: np.mean(group_embeddings[valid_clusters[valid_clusters[cluster_col] == cid].index], axis=0)
        for cid in valid_clusters[cluster_col].unique()
    }
    cids, vectors = list(centroids.keys()), np.array(list(centroids.values()))
    sim_matrix = cosine_similarity(vectors)
    
    merge_map = {}
    temp_id_counter = -2
    for i in range(len(cids)):
        for j in range(i + 1, len(cids)):
            if sim_matrix[i, j] >= merge_threshold:
                id1, id2 = cids[i], cids[j]
                g1, g2 = merge_map.get(id1, id1), merge_map.get(id2, id2)
                if g1 != g2:
                    new_id = min(g1, g2) if isinstance(g1, int) and g1 < 0 else temp_id_counter
                    for k, v in merge_map.items():
                        if v in [g1, g2]: merge_map[k] = new_id
                    merge_map.update({id1: new_id, id2: new_id})
                    if new_id == temp_id_counter: temp_id_counter -= 1
    
    if not merge_map: return df_group
    
    df_group[cluster_col] = df_group[cluster_col].apply(lambda x: merge_map.get(x, x) if x != -1 else x)
    merged_ids = sorted(df_group[(df_group[cluster_col] < 0) & (df_group[cluster_col] != -1)][cluster_col].unique())
    
    if merged_ids:
        max_id = df_group[df_group[cluster_col] >= 0][cluster_col].max() if not df_group[df_group[cluster_col] >= 0].empty else -1
        id_remap = {old: max_id + i + 1 for i, old in enumerate(merged_ids)}
        df_group[cluster_col] = df_group[cluster_col].replace(id_remap)
        
    return df_group

# 메인 실행 로직
def main(df):
    if os.path.exists(EMBEDDING_FILE_PATH):
        print(f"▶ 저장된 임베딩 파일 로드: '{EMBEDDING_FILE_PATH}'")
        embeddings = np.load(EMBEDDING_FILE_PATH)['embeddings']
    else:
        print("▶ 임베딩 생성...")
        model = SentenceTransformer("jhgan/ko-sbert-sts")
        embeddings = model.encode(df["content"].fillna("").tolist(), show_progress_bar=True)
        np.savez_compressed(EMBEDDING_FILE_PATH, embeddings=embeddings)
    print("▶ 임베딩 준비 완료.")

    print("\n단계 1: MAIN_NAME별 로컬 클러스터링, 병합, 재할당 시작")
    processed_groups = []
    
    for main_name, group in tqdm(df.groupby("predicted_MAIN_NAME"), desc="MAIN_NAME별 처리"):
        group = group.copy()
        original_indices = group.index.tolist()
        group.reset_index(drop=True, inplace=True)
        
        group_embeddings_slice = embeddings[original_indices]
        
        n_articles = len(group)
        min_cluster_size = max(2, min(int(n_articles * 0.002), 50))
        n_neighbors = max(5, min(min_cluster_size * 2, 50))
        min_samples = max(3, min(int(min_cluster_size * 0.5), 15))
        
        reduced = umap.UMAP(n_neighbors=n_neighbors, n_components=15, metric="cosine", random_state=42).fit_transform(group_embeddings_slice)
        group['cluster_id'] = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples).fit_predict(reduced)
        
        group = merge_clusters(group, group_embeddings_slice, MERGE_THRESHOLD)
        
        noise_mask = group['cluster_id'] == -1
        if noise_mask.sum() > 0 and (~noise_mask).sum() > 0:
            valid_clusters = group[~noise_mask]
            centroids = {cid: np.mean(group_embeddings_slice[valid_clusters[valid_clusters['cluster_id'] == cid].index], 0) for cid in valid_clusters['cluster_id'].unique()}
            cids, vectors = list(centroids.keys()), np.array(list(centroids.values()))
            
            for idx in group[noise_mask].index: 
                sims = cosine_similarity(group_embeddings_slice[idx].reshape(1, -1), vectors)[0]
                best = np.argmax(sims)
                if sims[best] >= REASSIGN_THRESHOLD:
                    assigned_cid = cids[best]
                    group.loc[idx, 'cluster_id'] = assigned_cid
        
        group.index = original_indices
        processed_groups.append(group)

    df_processed = pd.concat(processed_groups)
    print("--- 단계 1 완료 ---")

    print("\n단계 2: 최종 ID 및 이름 부여 시작")
    df_processed['cluster_name'] = ""
    valid_mask = df_processed['cluster_id'] != -1
    df_processed.loc[valid_mask, 'cluster_name'] = df_processed[valid_mask].apply(
        lambda row: f"{row['predicted_MAIN_NAME']}_{int(row['cluster_id'])}", axis=1)
    
    noise_mask = df_processed['cluster_id'] == -1
    if noise_mask.sum() > 0:
        noise_numbers = df_processed[noise_mask].groupby('predicted_MAIN_NAME').cumcount() + 1
        df_processed.loc[noise_mask, 'cluster_name'] = df_processed[noise_mask]['predicted_MAIN_NAME'] + '_noise_' + noise_numbers.astype(str)

    unique_cluster_names = sorted(df_processed[valid_mask]['cluster_name'].unique())
    name_to_id_map = {name: i for i, name in enumerate(unique_cluster_names)}
    df_processed['final_cluster_id'] = df_processed['cluster_name'].map(name_to_id_map).fillna(-1).astype(int)
    print("--- 단계 2 완료 ---")
    
    print("\n단계 3: 최종 결과 정리 및 저장 시작")
    final_reps_map = {}
    valid_clusters_final = df_processed[df_processed['final_cluster_id'] != -1]
    if not valid_clusters_final.empty:
         # 대표 문장을 계산할 때 content 사용
         final_reps_map = {
            cname: get_representative_doc(embeddings[g.index], g["content"].tolist()) 
            for cname, g in valid_clusters_final.groupby('cluster_name')
        }
    df_processed['representative'] = df_processed['cluster_name'].map(final_reps_map)
    df_processed.loc[df_processed['representative'].isnull(), 'representative'] = df_processed['content'] 

    final_noise_count = (df_processed['final_cluster_id'] == -1).sum()
    
    df_processed.drop(columns=['cluster_id', 'category'], inplace=True, errors='ignore')
    df_processed.rename(columns={
        'final_cluster_id': 'cluster_id', 
        'predicted_category': 'category',
        'predicted_MAIN_NAME': 'MAIN_NAME'
    }, inplace=True)
    
    df_processed = df_processed.sort_values(by=['MAIN_NAME', 'cluster_id'])
    
    final_cols = ['category', 'MAIN_NAME', 'date', 'cluster_id', 'cluster_name', 'representative', 'title', 'content', 'url']
    other_cols = [col for col in df_processed.columns if col not in final_cols and col != 'index']
    df_processed = df_processed[final_cols + other_cols]

    df_processed.to_csv(FINAL_OUTPUT_CSV, index=False, encoding="utf-8-sig")
    
    print(f"\n▶ 모든 작업이 완료되었습니다.")
    print(f"  - 최종 노이즈: {final_noise_count}개")
    print(f"  - 최종 결과가 '{FINAL_OUTPUT_CSV}' 파일에 저장되었습니다.")

if __name__ == "__main__":
    try:
        df_initial = pd.read_csv(INPUT_CSV_PATH)
        main(df_initial)
    except FileNotFoundError:
        print(f"오류: '{INPUT_CSV_PATH}' 파일을 찾을 수 없습니다.")

C:\Users\user\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


▶ 저장된 임베딩 파일 로드: '../../data/processed\full_article_embeddings.npz'
▶ 임베딩 준비 완료.

단계 1: MAIN_NAME별 로컬 클러스터링, 병합, 재할당 시작


MAIN_NAME별 처리:   0%|          | 0/17 [00:00<?, ?it/s]C:\Users\user\AppData\Roaming\Python\Python313\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
MAIN_NAME별 처리:   6%|▌         | 1/17 [00:28<07:40, 28.80s/it]C:\Users\user\AppData\Roaming\Python\Python313\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
C:\Users\user\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\depre

--- 단계 1 완료 ---

단계 2: 최종 ID 및 이름 부여 시작
--- 단계 2 완료 ---

단계 3: 최종 결과 정리 및 저장 시작

▶ 모든 작업이 완료되었습니다.
  - 최종 노이즈: 1640개
  - 최종 결과가 '../../data/processed\news_cluster.csv' 파일에 저장되었습니다.


In [ ]:
# 클러스터링 결과 확인용
import pandas as pd
FILE_PATH = "../data/processed/news_cluster.csv"
CLUSTER_ID_COL = "cluster_id"         # 노이즈(-1) 판별을 위해 클러스터 ID 컬럼 추가
CLUSTER_NAME_COL = "cluster_name"
TITLE_COL = "title"
REPRESENTATIVE_COL = "representative" 

try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"오류: '{FILE_PATH}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해주세요.")
    exit()
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
    exit()

# 필수 컬럼 존재 여부 확인
required_cols = [CLUSTER_ID_COL, CLUSTER_NAME_COL, TITLE_COL, REPRESENTATIVE_COL]
if not all(col in df.columns for col in required_cols):
    print(f"오류: 파일에 '{', '.join(required_cols)}' 컬럼이 모두 존재해야 합니다.")
    print(f"    현재 파일에 있는 컬럼: {df.columns.tolist()}")
    exit()

# 클러스터링 결과 요약 정보 출력
total_articles = len(df)
noise_articles_count = (df[CLUSTER_ID_COL] == -1).sum()
total_valid_clusters = df[df[CLUSTER_ID_COL] != -1][CLUSTER_NAME_COL].nunique()
noise_ratio = (noise_articles_count / total_articles) * 100 if total_articles > 0 else 0

print("="*50)
print("클러스터링 결과 전체 요약")
print(f"총 기사 수\t\t: {total_articles}개")
print(f"생성된 클러스터 수 (노이즈 제외)\t: {total_valid_clusters}개")
print(f"노이즈로 분류된 기사 수\t: {noise_articles_count}개")
print(f"노이즈 비율\t\t: {noise_ratio:.2f}%")
print("="*50)

# MAIN_NAME별 요약 정보 출력
print("\n" + "="*60)
print("MAIN_NAME별 상세 요약")

summary_list = []
for name, group in df.groupby('MAIN_NAME'):
    total = len(group)
    noise = (group[CLUSTER_ID_COL] == -1).sum()
    clusters = group[group[CLUSTER_ID_COL] != -1][CLUSTER_NAME_COL].nunique()
    ratio = (noise / total * 100) if total > 0 else 0
    summary_list.append({
        'MAIN_NAME': name,
        '총 기사 수': total,
        '클러스터 수': clusters,
        '노이즈 수': noise,
        '노이즈 비율 (%)': f"{ratio:.2f}"
    })

if summary_list:
    summary_df = pd.DataFrame(summary_list)
    # 총 기사 수가 많은 순서로 정렬
    summary_df = summary_df.sort_values(by='총 기사 수', ascending=False).reset_index(drop=True)
    print(summary_df.to_string()) 
else:
    print("요약할 데이터가 없습니다.")

print("="*60, "\n")

클러스터링 결과 전체 요약
총 기사 수		: 27969개
생성된 클러스터 수 (노이즈 제외)	: 1135개
노이즈로 분류된 기사 수	: 1640개
노이즈 비율		: 5.86%

MAIN_NAME별 상세 요약
         MAIN_NAME  총 기사 수  클러스터 수  노이즈 수 노이즈 비율 (%)
0        가상자산·규제정책    3653      40    462      12.65
1      법적 분쟁·정치 연루    3206      91    115       3.59
2          산업재해 사건    2825     107    148       5.24
3       제도·안전관리·정책    2085      90    149       7.15
4      시장·기업 관련 사건    2039      75     60       2.94
5    온라인·플랫폼 관련 이슈    1891      76    113       5.98
6        아동 학대·성범죄    1715      76     23       1.34
7     개인정보 유출·보호조치    1691      65     50       2.96
8       특검정치 연루 사건    1671      50     62       3.71
9       중대재해처벌법 기타    1415     103    176      12.44
10     금융사고·소비자 피해    1300      75     37       2.85
11      개인정보보호법 기타    1228      93     79       6.43
12     금융소비자보호법 기타     975      68     74       7.59
13     법·제도·사회적 이슈     906      53     52       5.74
14  아동 유기·방임·사망 사건     608      29      7       1.15
15          중대시민재해     437      15  

#### 클러스터링 이후 처리

In [ ]:
# LLM 이름 붙이기
import pandas as pd
import os
from openai import OpenAI

df = pd.read_csv("../../data/processed/news_cluster.csv")
#df = df.sample(n=20, random_state=42) # 테스트용
print(f"총 {len(df)}건 데이터 사용")

# OpenAI 클라이언트 세팅
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# GPT 요청 함수/프롬프트
def generate_event_name(representative: str, model: str = "gpt-4o-mini") -> str:
    if pd.isna(representative) or not str(representative).strip():
        return "대표 기사 없음"

    system_message = (
        "당신은 대표 기사를 보고, 사건을 대표하는 구체적인 '소분류' 이름을 생성하는 전문가입니다. "
        "소분류명은 5~20자 이내의 명사형으로, 사건의 주요 특징, '장소', 대상, 행위 등을 반영해야 합니다. "
        "너무 일반적이거나 포괄적인 단어는 피하고, 설명 없이 단어 또는 짧은 명사구만 출력하세요. "
        "장소가 있다면 반드시 포함하세요"
        "반드시 각 사건명은 고유해야 합니다."
    )

    user_message = (
        f"다음 대표 기사 내용을 바탕으로, 이 클러스터에 적합한 '소분류' 이름을 하나 생성해주세요.\n\n"
        f"## 대표 기사:\n{representative}\n\n"
        "조건:\n"
        "- 5~20자, 명사형 또는 짧은 명사구\n"
        "- 사건의 주요 특징, 대상, 장소, 행위 포함\n"
        "- 설명 없이 단어 또는 짧은 명사구만\n"
        "예시 출력:\n"
        "- 서울 강남 아파트 화재\n"
        "- 부산시 초등학생 학대사건\n"
        "- 의료기관 개인정보 유출\n"
        "- 이하늘 사이버 명예훼손 게시글\n"
        "이 형식에 맞게 생성해주세요:"
    )

    max_retries = 5
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": user_message},
                ],
                temperature=0.2,
                max_tokens=30,
                n=1,
            )
            return response.choices[0].message.content.strip().replace('"', '').replace("'", "")
        except Exception as e:
            if attempt < max_retries - 1:
                continue
            return f"[생성 실패: {e}]"


# 클러스터별 대표 기사로 소분류명 생성
cluster_names = {}
print("\n=== 클러스터 소분류명 생성 결과 ===")
for cluster_name, sub in df.groupby("cluster_name"):
    representative = sub["representative"].iloc[0]  # 대표 기사 1개 이용
    event_name = generate_event_name(representative)
    print(f"[{cluster_name}] → {event_name}")
    cluster_names[cluster_name] = event_name

df["cluster_event_name"] = df["cluster_name"].map(cluster_names)
df.to_csv("../../data/processed/news_cluster_w_event_name.csv", index=False, encoding="utf-8-sig")

총 20건 데이터 사용

=== 클러스터 소분류명 생성 결과 ===
[가상자산·규제정책_76] → 국회 상속세 개정 논의
[가상자산·규제정책_noise_274] → 국내 중장년층 가상화폐 투자 증가
[개인정보 유출·보호조치_159] → 예스24 랜섬웨어 해킹
[금융사고·소비자 피해_143] → 서울 대포통장 보이스피싱 조직
[금융소비자보호법 기타_107] → 동국대 청년 금융교육
[법적 분쟁·정치 연루_164] → 부산 전한길 내란선동 고발
[법적 분쟁·정치 연루_171] → 서울 강남 유튜버 협박 사건
[법적 분쟁·정치 연루_172] → 대구 불법 여론조사 사건
[법적 분쟁·정치 연루_6] → 서울구치소 성 상납 의혹
[산업재해 사건_194] → 태안화력 하청노동자 사망사고
[산업재해 사건_197] → 경기도 시흥 제빵공장 사고
[산업재해 사건_52] → 울산 앞바다 작업자 사망 사고
[시장·기업 관련 사건_161] → 서울 SM엔터 시세조종 사건
[아동 유기·방임·사망 사건_66] → 인천 초등학생 방임 화재
[온라인·플랫폼 관련 이슈_165] → 서울 개인정보 보호 소송
[온라인·플랫폼 관련 이슈_168] → 세종시 AI 보안관제 인증
[제도·안전관리·정책_153] → 중소기업 안전보건 지원사업
[제도·안전관리·정책_44] → 한국예탁결제원 중대재해 인증
[제도·안전관리·정책_52] → 서울 강남 노동정책 세미나
[중대시민재해_37] → 창원NC파크 구조물 추락 사고


In [ ]:
# LLM 이름붙인 결과 확인용 -> 월별 최다 사건명 count
import pandas as pd
from tqdm import tqdm

FILE_PATH ="../../data/processed/news_cluster_w_event_name.csv"
DATE_COL = "date"
MAIN_NAME_COL = "MAIN_NAME" 
CLUSTER_NAME_COL = "cluster_event_name"

try:
    df = pd.read_csv(FILE_PATH)
except FileNotFoundError:
    print(f"오류: '{FILE_PATH}' 파일을 찾을 수 없습니다. 파일 경로를 다시 확인해주세요.")
    exit()
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
    exit()

# 필수 컬럼 존재 여부 확인
required_cols = [DATE_COL, MAIN_NAME_COL, CLUSTER_NAME_COL]
if not all(col in df.columns for col in required_cols):
    print(f"오류: 파일에 '{', '.join(required_cols)}' 컬럼이 모두 존재해야 합니다.")
    print(f"    현재 파일에 있는 컬럼: {df.columns.tolist()}")
    exit()

# 데이터 전처리: 'year_month' 컬럼 생성
print("▶ 날짜 데이터를 처리하여 '연도-월' 정보를 추출합니다...")
# date 컬럼을 문자열로 변환하여 처리 (오류 방지)
df[DATE_COL] = df[DATE_COL].astype(str)
# 날짜 형식이 올바른 데이터만 필터링 (예: 14자리)
df = df[df[DATE_COL].str.len() >= 6].copy()
# 앞에서 6자리 (YYYYMM)를 잘라 'year_month' 컬럼 생성
df['year_month'] = df[DATE_COL].str[:6]
print("▶ 날짜 처리 완료.\n")


# 월별, MAIN_NAME별 최다 클러스터 분석
print(f"▶ 월별, '{MAIN_NAME_COL}'별 최다 발생 클러스터를 분석합니다...")
# 결과를 저장할 리스트
monthly_summary = []
# 'year_month'와 'MAIN_NAME'으로 그룹화
grouped = df.groupby(['year_month', MAIN_NAME_COL])
for (year_month, main_name), group in tqdm(grouped, desc="월별/카테고리별 분석 중"):
    # 각 그룹 내에서 cluster_event_name의 개수를 셈
    cluster_counts = group[CLUSTER_NAME_COL].value_counts()
    # 그룹에 데이터가 없는 경우 건너뛰기
    if cluster_counts.empty:
        continue
    # 가장 많이 발생한 클러스터의 이름과 횟수 추출
    top_cluster_name = cluster_counts.index[0]
    top_cluster_count = cluster_counts.iloc[0]
    # 해당 그룹의 전체 기사 수
    total_articles_in_group = len(group)
    
    monthly_summary.append({
        'Month': year_month,
        'MAIN_NAME': main_name,
        'Top Cluster Event': top_cluster_name,
        'Count': top_cluster_count,
        'Total Articles': total_articles_in_group
    })

# 최종 결과 출력
if not monthly_summary:
    print("\n분석할 데이터가 없습니다.")
else:
    summary_df = pd.DataFrame(monthly_summary)
    summary_df['Percentage (%)'] = (summary_df['Count'] / summary_df['Total Articles'] * 100).round(2)
    summary_df = summary_df.sort_values(by=['Month', 'MAIN_NAME'], ascending=[False, True])
    
    print("\n" + "="*100)
    print("📊 월별 / MAIN_NAME별 최다 발생 클러스터 이벤트 요약")

    print(summary_df.to_string(index=False))
    print("="*100)

✅ '../../data/processed/news_cluster_w_event_name.csv' 파일을 성공적으로 불러왔습니다. (총 20개 행)

▶ 날짜 데이터를 처리하여 '연도-월' 정보를 추출합니다...
▶ 날짜 처리 완료.

▶ 월별, 'MAIN_NAME'별 최다 발생 클러스터를 분석합니다...


월별/카테고리별 분석 중: 100%|██████████| 19/19 [00:00<00:00, 728.34it/s]


📊 월별 / MAIN_NAME별 최다 발생 클러스터 이벤트 요약
 Month      MAIN_NAME  Top Cluster Event  Count  Total Articles  Percentage (%)
202508 아동 유기·방임·사망 사건      인천 초등학생 방임 화재      1               1           100.0
202508     제도·안전관리·정책    한국예탁결제원 중대재해 인증      1               1           100.0
202507    시장·기업 관련 사건    서울 SM엔터 시세조종 사건      1               1           100.0
202507     제도·안전관리·정책     서울 강남 노동정책 세미나      1               1           100.0
202506   개인정보 유출·보호조치       예스24 랜섬웨어 해킹      1               1           100.0
202506    법적 분쟁·정치 연루      서울구치소 성 상납 의혹      1               1           100.0
202506        산업재해 사건     경기도 시흥 제빵공장 사고      1               1           100.0
202505    금융소비자보호법 기타        동국대 청년 금융교육      1               1           100.0
202505        산업재해 사건   울산 앞바다 작업자 사망 사고      1               1           100.0
202505  온라인·플랫폼 관련 이슈      서울 개인정보 보호 소송      1               1           100.0
202504    금융사고·소비자 피해   서울 대포통장 보이스피싱 조직      1               1           100.0
202